In [20]:
import pandas as pd
import numpy as np
from pathlib import Path

In [21]:
def extract_data(source):
    """
    Load multiple excel files into one pandas dataframe
    :param source: source path
    :return: dataframe
    """

    # read files
    path = Path(source)
    # create empty dataframe
    df = pd.DataFrame()
    # iterate all files
    for file in path.iterdir():
        df = df.append(pd.read_excel(file, engine="openpyxl"))

    return df

In [22]:
def transform_data(df):
    """
    filter only GBK data.
    :param df: main dataframe
    :return dataframe
    """
    df = df.loc[df["Franchisee"].str.contains("GBK")]
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "_")

    return df

In [23]:
df = extract_data("../data/")
df.shape

(97758, 43)

In [24]:
gbk = transform_data(df)
gbk.shape

(54201, 43)

In [25]:
gbk.columns = gbk.columns.str.replace(" ", "_").lower()
gbk.head()

AttributeError: 'Index' object has no attribute 'lower'

In [8]:
if __name__ == "__main__":
    # extract data
    source_data = "../data/"
    df = extract_data(source_data)
    df = transform_data(df)
    print(df.shape)
    

(54201, 43)
